In [9]:
# install dependencies
%pip install -r ../requirements.txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
# setup
import os
import pandas as pd
import tmdbsimple as tmdb
from typing import Dict
from IPython.display import display


tmdb.API_KEY = os.environ["TMDB_API_KEY"]

KeyError: 'TMDB_API_KEY'

In [ ]:
# read file
movie_list = pd.read_csv('../data/movie_list.csv')

watched_movies = movie_list.loc[movie_list.Completed.notnull()]

display(watched_movies[["Name", "imdb"]])

print(watched_movies.dtypes)

,Name,imdb
0,Us,https://www.imdb.com/title/tt6857112/
1,Annabelle,https://www.imdb.com/title/tt3322940/
2,Up in the Air,https://www.imdb.com/title/tt1193138/
3,The Conjuring,https://www.imdb.com/title/tt1457767/
4,A Quiet Place,https://imdb.com/title/tt6644200/
...,...,...
104,Once Upon A Time in Hollywood,https://www.imdb.com/title/tt7131622/
105,The Girl With the Dragon Tattoo,https://www.imdb.com/title/tt1568346/
106,Nimona,https://www.imdb.com/title/tt19500164/
107,Spy Kids,https://www.imdb.com/title/tt0227538/


Name                   object
Status                 object
Completed              object
Collin Rating         float64
Valerie Rating        float64
Genre                  object
Source                 object
Previously Seen By     object
Chosen By              object
imdb                   object
Release Date           object
dtype: object


In [ ]:
# helper func, get imdb id from link
def imdb_link_to_id(imdb_link: str) -> str:
    return imdb_link.split("/")[-2]

print(imdb_link_to_id("https://imdb.com/title/tt6644200/"))

tt6644200


In [ ]:
watched_movies["imdb_id"] = watched_movies["imdb"].apply(imdb_link_to_id)

display(watched_movies[["Name", "imdb_id"]])

/var/folders/13/20wdg7_5643f5n2x2h6pm7n00000gn/T/ipykernel_16802/2658727111.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  watched_movies["imdb_id"] = watched_movies["imdb"].apply(imdb_link_to_id)


,Name,imdb_id
0,Us,tt6857112
1,Annabelle,tt3322940
2,Up in the Air,tt1193138
3,The Conjuring,tt1457767
4,A Quiet Place,tt6644200
...,...,...
104,Once Upon A Time in Hollywood,tt7131622
105,The Girl With the Dragon Tattoo,tt1568346
106,Nimona,tt19500164
107,Spy Kids,tt0227538


In [ ]:
# get extra info from tmdb
def get_movie_info(imdb_id: str) -> Dict:
    found_data: Dict = tmdb.Find(imdb_id).info(external_source="imdb_id")["movie_results"][0]
    return {
        "tmdb_id": found_data.get("id", -1),
        "summary": found_data.get("overview", ""),
        "adult": found_data.get("adult", False),
        "genre_ids": found_data.get("genre_ids", []),
        "popularity": found_data.get("popularity"),
        "vote_average": found_data.get("vote_average"),
        "vote_count": found_data.get("vote_count"),
        "budget": found_data.get("budget"),
        "runtime": found_data.get("runtime")
    }

print(get_movie_info("tt3322940"))

{'tmdb_id': 250546, 'summary': 'A couple begins to experience terrifying supernatural occurrences involving a vintage doll shortly after their home is invaded by satanic cultists.', 'adult': False, 'genre_ids': [27], 'popularity': 64.794, 'vote_average': 5.77, 'vote_count': 6044, 'budget': None, 'runtime': None}


In [ ]:
# enhance with extra info from tmdb
watched_movies["extra_info"] = watched_movies["imdb_id"].map(get_movie_info)

display(watched_movies[["Name", "extra_info"]])

/var/folders/13/20wdg7_5643f5n2x2h6pm7n00000gn/T/ipykernel_16802/2275995514.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  watched_movies["extra_info"] = watched_movies["imdb_id"].map(get_movie_info)


,Name,extra_info
0,Us,"{'tmdb_id': 458723, 'summary': 'Husband and wi..."
1,Annabelle,"{'tmdb_id': 250546, 'summary': 'A couple begin..."
2,Up in the Air,"{'tmdb_id': 22947, 'summary': 'Corporate downs..."
3,The Conjuring,"{'tmdb_id': 138843, 'summary': 'Paranormal inv..."
4,A Quiet Place,"{'tmdb_id': 447332, 'summary': 'A family is fo..."
...,...,...
104,Once Upon A Time in Hollywood,"{'tmdb_id': 466272, 'summary': 'Los Angeles, 1..."
105,The Girl With the Dragon Tattoo,"{'tmdb_id': 65754, 'summary': 'Disgraced journ..."
106,Nimona,"{'tmdb_id': 961323, 'summary': 'A knight frame..."
107,Spy Kids,"{'tmdb_id': 10054, 'summary': 'Carmen and Juni..."


In [ ]:
# explode extra_info out into unique columns
def getColumn(df: pd.DataFrame, column_name: str) -> pd.DataFrame:
    df[column_name] = df["extra_info"].map(lambda x: x[column_name])
    return df
watched_movies = getColumn(watched_movies, "tmdb_id")
watched_movies = getColumn(watched_movies, "summary")
watched_movies = getColumn(watched_movies, "adult")
watched_movies = getColumn(watched_movies, "genre_ids")
watched_movies = getColumn(watched_movies, "popularity")
watched_movies = getColumn(watched_movies, "vote_average")
watched_movies = getColumn(watched_movies, "vote_count")
watched_movies = getColumn(watched_movies, "budget")
watched_movies = getColumn(watched_movies, "runtime")

display(watched_movies[["Name", "tmdb_id", "summary", "adult", "genre_ids", "popularity", "vote_average", "vote_count", "budget", "runtime"]])

/var/folders/13/20wdg7_5643f5n2x2h6pm7n00000gn/T/ipykernel_16802/596358465.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = df["extra_info"].map(lambda x: x[column_name])
/var/folders/13/20wdg7_5643f5n2x2h6pm7n00000gn/T/ipykernel_16802/596358465.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = df["extra_info"].map(lambda x: x[column_name])
/var/folders/13/20wdg7_5643f5n2x2h6pm7n00000gn/T/ipykernel_16802/596358465.py:3: SettingWithCopyWarning: 
A value is trying to b

,Name,tmdb_id,summary,adult,genre_ids,popularity,vote_average,vote_count,budget,runtime
0,Us,458723,Husband and wife Gabe and Adelaide Wilson take...,False,"[27, 53]",35.044,7.000,7257,None,None
1,Annabelle,250546,A couple begins to experience terrifying super...,False,[27],64.794,5.770,6044,None,None
2,Up in the Air,22947,Corporate downsizing expert Ryan Bingham spend...,False,"[18, 10749]",19.648,6.900,3575,None,None
3,The Conjuring,138843,Paranormal investigators Ed and Lorraine Warre...,False,"[27, 53]",143.517,7.500,11437,None,None
4,A Quiet Place,447332,A family is forced to live in silence while hi...,False,"[27, 18, 878]",96.357,7.400,14058,None,None
...,...,...,...,...,...,...,...,...,...,...
104,Once Upon A Time in Hollywood,466272,"Los Angeles, 1969. TV star Rick Dalton, a stru...",False,"[35, 18, 53]",58.632,7.439,13319,None,None
105,The Girl With the Dragon Tattoo,65754,Disgraced journalist Mikael Blomkvist investig...,False,"[53, 80, 9648]",28.766,7.374,6909,None,None
106,Nimona,961323,A knight framed for a tragic crime teams with ...,False,"[12, 16, 14, 10751, 28, 878]",34.169,7.907,1061,None,None
107,Spy Kids,10054,Carmen and Juni think their parents are boring...,False,"[28, 35, 10751, 12]",27.216,5.796,3394,None,None


In [ ]:
# write out modified data to use elsewhere
watched_movies.to_csv("../data/enhanced_movie_list.csv", index=False)

NameError: name 'watched_movies' is not defined